<h2>Preprocessing, Yay!</h2>

In [1]:
#importing dependencies
import pandas as pd
import numpy as np
import seaborn as sns
import re
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
#!pip install wordcloud
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay

In [2]:
df = pd.read_csv('labeled_data.csv')
df

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...,...,...,...,...,...
24778,25291,3,0,2,1,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779,25292,3,0,1,2,2,"you've gone and broke the wrong heart baby, an..."
24780,25294,3,0,3,0,1,young buck wanna eat!!.. dat nigguh like I ain...
24781,25295,6,0,6,0,1,youu got wild bitches tellin you lies


In [3]:
print(df['tweet'].iloc[0], "\n")
print(df['tweet'].iloc[1], "\n")
print(df['tweet'].iloc[2], "\n")
print(df['tweet'].iloc[3], "\n")
print(df['tweet'].iloc[4], "\n")

!!! RT @mayasolovely: As a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out... 

!!!!! RT @mleew17: boy dats cold...tyga dwn bad for cuffin dat hoe in the 1st place!! 

!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby4life: You ever fuck a bitch and she start to cry? You be confused as shit 

!!!!!!!!! RT @C_G_Anderson: @viva_based she look like a tranny 

!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you hear about me might be true or it might be faker than the bitch who told it to ya &#57361; 



In [4]:
#defining a function to clean the tweets' text
def clean(text):
    text = str(text).lower()
    text = re.sub(r"https?://\S+|www\.\S+", "", text, flags = re.MULTILINE)
    test = re.sub(r"\@w+|\#", "", text)
    text = re.sub(r"[^\w\s]", "", text)
    tweet_tokens = word_tokenize(text)
    text = [word for word in text.split(" ") if word not in stop_words]
    text = " ".join(text)
    return text

df.tweet = df["tweet"].apply(clean)
df.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,rt mayasolovely woman shouldnt complain clean...
1,1,3,0,3,0,1,rt mleew17 boy dats coldtyga dwn bad cuffin d...
2,2,3,0,3,0,1,rt urkindofbrand dawg rt 80sbaby4life ever fu...
3,3,3,0,2,1,1,rt c_g_anderson viva_based look like tranny
4,4,6,0,6,0,1,rt shenikaroberts shit hear might true might ...


In [5]:
lemmatizer = WordNetLemmatizer()
def lemmatizing(data):
    tweet = [lemmatizer.lemmatize(w) for w in data]
    return data

In [6]:
df['tweet'] = df['tweet'].apply(lambda x: lemmatizing(x))

In [7]:
df['tweet']

0         rt mayasolovely woman shouldnt complain clean...
1         rt mleew17 boy dats coldtyga dwn bad cuffin d...
2         rt urkindofbrand dawg rt 80sbaby4life ever fu...
3              rt c_g_anderson viva_based look like tranny
4         rt shenikaroberts shit hear might true might ...
                               ...                        
24778    yous muthafin lie 8220lifeasking 20_pearls cor...
24779    youve gone broke wrong heart baby drove rednec...
24780    young buck wanna eat dat nigguh like aint fuck...
24781                    youu got wild bitches tellin lies
24782    ruffled  ntac eileen dahlia  beautiful color c...
Name: tweet, Length: 24783, dtype: object

In [8]:
df['class'].value_counts()

class
1    19190
2     4163
0     1430
Name: count, dtype: int64

<h2>Creating a Bigram language model </h2>

In [9]:
#initialising the vectoriser

vect = TfidfVectorizer(ngram_range=(1,2)).fit(df['tweet'])

In [10]:
feature_names = vect.get_feature_names_out()
print("Number of features: {} \n".format(len(feature_names)))
print("First 20 features: \n{}".format(feature_names[0:20]))

Number of features: 184382 

First 20 features: 
['007' '007beardownjedi' '007beardownjedi afl' '007hertzrumble'
 '007hertzrumble via' '007m_h' '007m_h lilduval' '00_jackie'
 '00_jackie darknight420' '00_jackie wanna' '00sexilexi00'
 '00sexilexi00 bitch' '00sexilexi00 freeze' '00sexilexi00 nigga'
 '00sexilexi00 omg' '00sexilexi00 socass_' '00sexilexi00 swiggety' '01'
 '01 going' '01 loss']


<h2>Trigram Language Model

In [11]:
#initialising the vectoriser

vect2 = TfidfVectorizer(ngram_range=(1,3)).fit(df['tweet'])

In [12]:
feature_names = vect2.get_feature_names_out()
print("Number of features: {} \n".format(len(feature_names)))
print("First 20 features: \n{}".format(feature_names[0:20]))

Number of features: 347639 

First 20 features: 
['007' '007beardownjedi' '007beardownjedi afl'
 '007beardownjedi afl american' '007hertzrumble' '007hertzrumble via'
 '007hertzrumble via wordpressdotcom' '007m_h' '007m_h lilduval'
 '007m_h lilduval damn' '00_jackie' '00_jackie darknight420'
 '00_jackie darknight420 allahthefairy' '00_jackie wanna'
 '00_jackie wanna play' '00sexilexi00' '00sexilexi00 bitch'
 '00sexilexi00 bitch ass' '00sexilexi00 freeze'
 '00sexilexi00 freeze bitch']


<h2> Building the Logistic Regression Model</h2>

In [13]:
X = df['tweet']
Y = df['class']

X = vect2.transform(X)

In [14]:
X

<24783x347639 sparse matrix of type '<class 'numpy.float64'>'
	with 580093 stored elements in Compressed Sparse Row format>

In [15]:
#oversampling the existing data with ADASYN
from imblearn.over_sampling import ADASYN

# Assuming you have your imbalanced data as X (features) and y (labels)
X_resampled, y_resampled = ADASYN().fit_resample(X, Y)

# Now X_resampled and y_resampled contain the oversampled data


In [16]:
#dividing the data into training and testing data
x_train, x_test, y_train, y_test = train_test_split(X_resampled,y_resampled, test_size = 0.2, random_state = 42)

In [17]:
y_train.shape

(45904,)

In [18]:
y_test.shape

(11476,)

In [19]:
x_test

<11476x347639 sparse matrix of type '<class 'numpy.float64'>'
	with 382309 stored elements in Compressed Sparse Row format>

<h2> Guess I'll have to oversample, too. Fun.</h2>

In [20]:
#Building the Logistic Regression Model
logreg = LogisticRegression(penalty = 'l2', C = 1.0)
logreg.fit(x_train, y_train)
logreg_predict = logreg.predict(x_test)
logreg_acc = accuracy_score(logreg_predict, y_test)
print("The model's accuracy is {: .2f}".format(logreg_acc*100))

The model's accuracy is  97.15


In [21]:
print(confusion_matrix(y_test, logreg_predict))
print("\n")
print(classification_report(y_test, logreg_predict))


[[3882    0    0]
 [ 127 3514  194]
 [   0    6 3753]]


              precision    recall  f1-score   support

           0       0.97      1.00      0.98      3882
           1       1.00      0.92      0.96      3835
           2       0.95      1.00      0.97      3759

    accuracy                           0.97     11476
   macro avg       0.97      0.97      0.97     11476
weighted avg       0.97      0.97      0.97     11476



<h2>Hyperparameter Tuning</h2>

In [22]:
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')

In [57]:
param_grid = {'C':[100, 10, 1.0, 0.1, 0.01], 'solver':['newton-cg', 'lbfgs', 'liblinear']}
grid = GridSearchCV(LogisticRegression(penalty='l2', C = 0.5), param_grid, cv = 5)
grid.fit(x_train, y_train)
print("Best cross-validation score: {:.2f}\n".format(grid.best_score_))
print("Best parameters: {}".format(grid.best_params_))

Best cross-validation score: 0.98

Best parameters: {'C': 100, 'solver': 'newton-cg'}


In [58]:
y_pred = grid.predict(x_test)

In [59]:
logreg_acc2 = accuracy_score(y_pred, y_test)
print("Test accuracy: {:.2f}%".format(logreg_acc2*100))

Test accuracy: 98.16%


In [60]:
print(confusion_matrix(y_test, y_pred))
print("\n")
print(classification_report(y_test, y_pred))

[[3882    0    0]
 [  72 3629  134]
 [   0    5 3754]]


              precision    recall  f1-score   support

           0       0.98      1.00      0.99      3882
           1       1.00      0.95      0.97      3835
           2       0.97      1.00      0.98      3759

    accuracy                           0.98     11476
   macro avg       0.98      0.98      0.98     11476
weighted avg       0.98      0.98      0.98     11476



<h2>Testing the model</h2>

In [114]:
input_data = lemmatizing(clean("That's a fruit, idiot."))
input_data

'thats fruit idiot'

In [115]:
input_text = vect2.transform(input_data.split(" "))
input_text

<3x347639 sparse matrix of type '<class 'numpy.float64'>'
	with 3 stored elements in Compressed Sparse Row format>

In [116]:
logreg.predict(input_text)

array([1, 1, 2])

In [117]:
grid.predict(input_text)

array([1, 1, 1])